In [ ]:
# -*- coding: utf-8 -*-
"""submission_script.ipynb"""

# 데이터 분석 및 시각화를 위한 라이브러리 임포트
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# 1. 데이터 로드 및 병합
print("데이터를 불러오고 병합합니다...")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
building_info_df = pd.read_csv('building_info.csv', encoding='utf-8')
train_df = pd.merge(train_df, building_info_df, on='건물번호', how='left')
test_df = pd.merge(test_df, building_info_df, on='건물번호', how='left')

# 2. 기본 전처리 및 파생변수 생성
print("기본 전처리 및 파생변수 생성을 시작합니다...")
for df in [train_df, test_df]:
    cols_to_process = ['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']
    for col in cols_to_process:
        df[col] = df[col].replace('-', '0')
        df[col] = pd.to_numeric(df[col])
    df['일시'] = pd.to_datetime(df['일시'], format='%Y%m%d %H')
    df['month'] = df['일시'].dt.month
    df['day'] = df['일시'].dt.day
    df['hour'] = df['일시'].dt.hour
    df['weekday'] = df['일시'].dt.weekday
    df['냉방면적비율'] = df['냉방면적(m2)'] / (df['연면적(m2)'] + 1e-6)
    holidays = ['2024-06-06', '2024-08-15']
    df['is_holiday'] = ((df['weekday'] >= 5) | (df['일시'].dt.strftime('%Y-%m-%d').isin(holidays))).astype(int)

# 3. 모델 학습 및 예측
print("\n--- 모델 학습 및 예측 ---")
default_params = {
    'objective': 'regression_l1',
    'metric': 'rmse',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1,
    'boosting_type': 'gbdt'
}

# 처 목록
features_to_exclude = ['num_date_time', '일시', '전력소비량(kWh)', '일조(hr)', '일사(MJ/m2)', '건물유형']
features = [col for col in train_df.columns if col not in features_to_exclude]
target = '전력소비량(kWh)'

# 건물 유형별 모델 학습 및 예측
building_types = train_df['건물유형'].unique()
final_predictions = []

for b_type in tqdm(building_types, desc="모델 학습 및 예측 진행"):
    train_type_df = train_df[train_df['건물유형'] == b_type]
    test_type_df = test_df[test_df['건물유형'] == b_type]

    X_train_full = train_type_df[features]
    y_train_full = train_type_df[target]
    X_test = test_type_df[features]

    if not X_test.empty:
        final_model = lgb.LGBMRegressor(**default_params)
        final_model.fit(X_train_full, y_train_full)
        test_preds = final_model.predict(X_test)

        # 예측값이 음수일 경우 0으로 보정
        test_preds[test_preds < 0] = 0

        submission_part = test_type_df[['num_date_time']].copy()
        submission_part['answer'] = test_preds
        final_predictions.append(submission_part)

# 최종 제출 파일 병합 및 생성
submission_df = pd.concat(final_predictions, ignore_index=True)
submission_df = submission_df.sort_values(by='num_date_time').reset_index(drop=True)

# 4. 제출 파일 저장
print("\nsubmission.csv 파일을 생성합니다...")
submission_df.to_csv('submission.csv', index=False)
print("submission.csv 파일이 성공적으로 생성되었습니다!")

데이터를 불러오고 병합합니다...
기본 전처리 및 파생변수 생성을 시작합니다...

--- 모델 학습 및 예측 ---


모델 학습 및 예측 진행: 100%|██████████| 10/10 [00:26<00:00,  2.64s/it]


submission.csv 파일을 생성합니다...
submission.csv 파일이 성공적으로 생성되었습니다!


In [ ]:
!pip freeze > requirements.txt